In [1]:
#directions = [5,15,20,25,35]
#labels = ['extremeleft','left','middle','right','extremeright']

directions = [0,5,10,15,20,25,30,35]
labels = ['baseline','5','10','15','middle','25','30','extremeright']

labeldict = dict(zip(directions,labels))
n_each = 20 #15

In [2]:
import random
import pandas as pd
import numpy as np
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/anaconda3/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['random']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [3]:
def create_random_design():
    dirs = []
    lastdir = None
    for _ in range(n_each):
        acceptable = False
        while not acceptable:
            d = directions[:]
            random.shuffle(d)
            acceptable = d[0]!=lastdir
        lastdir=d[-1]
        dirs+=d
    return dirs

In [4]:
dirs = create_random_design()
" ".join([str(d) for d in dirs ])

'20 30 5 15 35 0 25 10 0 15 35 10 20 25 5 30 10 35 20 25 30 15 5 0 25 15 30 0 10 35 5 20 25 30 35 0 15 10 20 5 30 15 0 10 20 25 35 5 10 30 20 0 15 5 35 25 35 10 30 15 0 5 20 25 0 35 5 25 30 15 10 20 35 20 0 15 5 10 25 30 25 35 0 10 30 5 15 20 0 35 25 5 20 15 30 10 20 10 30 25 35 0 5 15 5 20 35 15 10 25 30 0 30 25 5 35 0 10 15 20 25 30 20 35 15 5 0 10 25 0 15 5 10 35 30 20 15 25 35 10 20 0 30 5 20 10 15 0 30 5 35 25 5 10 0 35 20 25 15 30'

In [5]:
len(dirs)

160

## What are the carry-over effects?

In [6]:
def get_transitions(dirs):
    # Get the first-order transition matrix
    trans = [ (dirs[i],dirs[i+1]) for i in range(len(dirs)-1) ]
    n = len(directions)
    transcount = np.zeros((n,n))
    for (d0,d1) in trans:
        d0i = directions.index(d0)
        d1i = directions.index(d1)
        transcount[d0i,d1i]+=1
    return transcount

In [7]:
transcount = get_transitions(dirs)
transcount

array([[0., 2., 5., 5., 0., 2., 3., 3.],
       [2., 0., 4., 3., 5., 1., 2., 3.],
       [2., 0., 0., 2., 6., 3., 4., 3.],
       [3., 6., 3., 0., 2., 1., 3., 2.],
       [4., 1., 2., 2., 0., 7., 1., 3.],
       [2., 4., 1., 2., 0., 0., 6., 5.],
       [2., 4., 2., 4., 3., 3., 0., 1.],
       [5., 3., 3., 2., 3., 3., 1., 0.]])

## Batch-generate schedules and pick the best one
Generations of scholars will wonder why on Earth we do this, but hey...

In [8]:
def badness(mat):
    # Tell us how bad this transition matrix is - how unequal are the transitional probability distributions?
    
    # Compute the average of each row
    mat = np.array(mat)
    n,_ = mat.shape
    # Restrict to the off-diagonal elements 
    rowvalues = [ vals[sel] for (vals,sel) in zip(mat,(1-np.eye(n)).astype(bool)) ]
    
    # Compute the deviation of each item from the average for that row
    dists = [ [ abs(r-np.mean(row)) for r in row ] for row in rowvalues ]
    
    return sum(np.array(dists).flatten())

In [9]:
badness(transcount)

68.0

In [10]:
class KeepTheBest:

    # Class Attribute
    badness = Inf
    design = None

    def consider(self,newbadness,newdesign):
        if newbadness<self.badness:
            self.badness = newbadness
            self.design = newdesign
        
    def print(self):
        print(self.badness)
        print(self.design)

In [ ]:
#design
#from functools import partial

keepthebest = KeepTheBest()
keepthebest.print()

def generate_random(i):
    #print(i)
    design = create_random_design()
    trans = get_transitions(design)
    howbad = badness(trans)
    keepthebest.consider(howbad,design)
    return 

N_DESIGNS = 1000
N_PARALLEL = 3

from multiprocessing import Pool, Value, Array


t0 = time.time()

p = Pool(N_PARALLEL)
res = p.map(generate_random, range(N_DESIGNS))

print("This took {} sec".format(time.time()-t0))

#print(keepthebest)
keepthebest.print()

In [ ]:
import multiprocessing
import ctypes
import numpy as np
from IPython.display import display, clear_output

N_DESIGNS = 100000000
N_PARALLEL = 3

shared_array_base = multiprocessing.Array(ctypes.c_double, N_DESIGNS)
shared_array = np.ctypeslib.as_array(shared_array_base.get_obj())
#shared_array = shared_array.reshape(10, 10)


shared_design_base = multiprocessing.Array(ctypes.c_int, n_each*len(directions))
shared_design = np.ctypeslib.as_array(shared_design_base.get_obj())

bestbad = multiprocessing.Value('d', 999999)


def generate_random(i,allbadness=shared_array,bestdesign=shared_design,bestb=bestbad):
    #print(i)
    design = create_random_design()
    trans = get_transitions(design)
    howbad = badness(trans)
    allbadness[i]=howbad
    if howbad<bestb.value:
        bestb.value=howbad
        for j,v in enumerate(design):
            shared_design[j]=v
    return 

# Parallel processing
#def my_func(i, def_param=shared_array):
#    shared_array[i,:] = i

t0 = time.time()

pool = multiprocessing.Pool(processes=N_PARALLEL)
pool.map(generate_random, range(N_DESIGNS))

print("This took {} sec".format(time.time()-t0))

#print(shared_array)
#bestbad_posthoc = min(shared_array)
#print(bestbad_posthoc,bestbad.value)

In [11]:
import seaborn as sns

In [12]:
sns.distplot(shared_array)
axvline(x=bestbad.value,color='red')
sns.despine(offset=5)
print(bestbad.value)

NameError: name 'shared_array' is not defined

In [ ]:
t0 = time.time()
#N_DESIGNS = 100000000
N_DESIGNS = 50000
#designs = []
badnesses = zeros(N_DESIGNS)

for i in range(N_DESIGNS):
    badnesses[i]=howbad
    if howbad<badnessbest:
        bestdesign=design
        badnessbest=howbad


In [ ]:
b = [ bad for _,bad in designs ]

In [ ]:
hist(b)

In [ ]:
best = min(b)
print(best)

In [ ]:
chosen,_ = designs[ b.index(best) ]

In [ ]:
trans = get_transitions(chosen)

In [ ]:
trans

## Output

In [ ]:
tab = pd.DataFrame({'direction':chosen})
tab['type']=[ labeldict[d] for d in chosen ]
tab['trial']=np.arange(len(chosen))+1
#tab.to_csv('recognitionyesno_theOne.csv')
tab.to_csv('recognitionyesno160.csv')

In [ ]:
tab

# Doing this the smart way (maybe)

In [ ]:
if True:
    #Highest number in square
    order_of_sq = int(input("Enter order of sq: "))

    #Number you want to start the square with
    top_left = int(input("Enter top left number: "))

    #Sets a placeholder for a variable called top_left_init
    top_left_init=0

    #Sets the initial value of top_left to a new variable because the code will change the value of top left later on 
    top_left_init += top_left

    #Initialize the value of count
    count = 0

    #Add 1 to the highest value in latin square to account for the range function (the ending number is always one less than the number you enter into the range function)
    for values in range (1,order_of_sq+1):

        #Prevents the program from adding too many characters to the line
        while count != order_of_sq:

            #Prints numbers with spaces after them in a horizontal manner
            print(top_left,sep=' ',end=' ')

            #Adds 1 to the top_left
            top_left += 1

            #Count is used to keep track of how many characters are in your line
            count+=1

            #Restarts the numbers in your line when you reach the highest number
            if top_left == order_of_sq+1:
                top_left = 1

        #Creates a new row
        print()
        count = 0

        #Calls the initial value of top_left and adds 1 to it before beginning the next row
        top_left_init += 1

        #Resets top_left_init to 1 if it reaches the highest number in the square
        if top_left_init == order_of_sq + 1:
            top_left_init = 1
            top_left = top_left_init
        else:
            top_left = top_left_init